In [ ]:
import sys
!{sys.executable} -m pip install google-api-python-client

In [ ]:
import csv

from googleapiclient.discovery import build

api_key = 'HIDDEN'  # The API key is hidden here and should be replaced before running the script. 
service = build('youtube', 'v3', developerKey=api_key)
nextPageToken = None

with open('seven.csv', 'w', newline='', encoding='utf-8') as results:  # Due to the utf8-encoding emojis can also be displayed.
    csvwrite = csv.writer(results)
    csvwrite.writerow(['Video ID', 'Title', 'Channel', 'Published At', 'Views','Likes'])
    
    while True:
        # This API requests requests the next 50 result videos for the search 'tradwife'.
        search_request = service.search().list(
            q='tradwife',
            part='snippet',
            type='video',
            maxResults=50,
            pageToken=nextPageToken
        )
        
        search_response = search_request.execute()

        video_ids = [item['id']['videoId'] for item in search_response.get('items', [])]
        if not video_ids:
            break

        # video_requests requests the statistics for the result videos to get the number of views and likes.
        video_request = service.videos().list(
            part='statistics,snippet',
            id=','.join(video_ids)
        )

        video_response = video_request.execute()

        # All wanted information of the result videos is saved in a csv-file.
        for item in video_response.get('items', []):
            video_id = item['id']
            title = item['snippet']['title']
            channel = item['snippet']['channelTitle']
            published = item['snippet']['publishedAt']
            views = item.get('statistics', {}).get('viewCount','0')
            likes = item.get('statistics', {}).get('likeCount', '0')
            csvwrite.writerow([video_id, title, channel, published, views, likes])

        nextPageToken = search_response.get('nextPageToken')  # This token makes sure the next 50 results are requested in the next search_request.

        if not nextPageToken: 
            break  # This loop breaks if all results for the term 'tradwife' are found. 